# Regional Load Forecasting Workshop
## การพยากรณ์โหลดไฟฟ้ารายพื้นที่

### Workshop Overview
This workshop covers end-to-end machine learning for electrical load forecasting, including:
- Data exploration and visualization
- Feature engineering
- Data imputation and preprocessing
- Model training and validation
- Performance evaluation
- Inference with new data

### Learning Objectives
- Understand electrical load patterns and seasonality
- Apply time series analysis techniques
- Build predictive models for load forecasting
- Evaluate model performance with appropriate metrics

## 1. Environment Setup and Data Import

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Machine Learning libraries
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer

# Time series libraries
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print("Libraries imported successfully!")

## 2. Generate Synthetic Electrical Load Data
Since we don't have real data, we'll create realistic synthetic data that mimics actual electrical load patterns.

In [2]:
def generate_electrical_load_data(start_date='2022-01-01', end_date='2024-01-01', freq='H'):
    """
    Generate synthetic electrical load data with realistic patterns
    """
    # Create date range
    dates = pd.date_range(start=start_date, end=end_date, freq=freq)
    n_samples = len(dates)
    
    # Base load (MW)
    base_load = 1000
    
    # Seasonal patterns
    # Annual seasonality (summer peaks due to AC usage)
    annual_pattern = 200 * np.sin(2 * np.pi * dates.dayofyear / 365.25 - np.pi/2)
    
    # Weekly seasonality (weekday vs weekend)
    weekly_pattern = 150 * (dates.weekday < 5).astype(int)  # Higher on weekdays
    
    # Daily seasonality (peak hours)
    daily_pattern = 300 * np.sin(2 * np.pi * dates.hour / 24 - np.pi/2) + \
                   200 * np.sin(4 * np.pi * dates.hour / 24)  # Two peaks per day
    
    # Weather influence (temperature proxy)
    temp_base = 25 + 10 * np.sin(2 * np.pi * dates.dayofyear / 365.25)
    temp_variation = np.random.normal(0, 5, n_samples)
    temperature = temp_base + temp_variation
    
    # Load increases with extreme temperatures (heating/cooling)
    temp_load = 10 * np.abs(temperature - 22)  # Comfortable temperature is 22°C
    
    # Random noise and special events
    noise = np.random.normal(0, 50, n_samples)
    
    # Calculate total load
    total_load = base_load + annual_pattern + weekly_pattern + daily_pattern + temp_load + noise
    
    # Ensure non-negative values
    total_load = np.maximum(total_load, 100)
    
    # Create humidity data
    humidity = 60 + 20 * np.sin(2 * np.pi * dates.dayofyear / 365.25) + np.random.normal(0, 10, n_samples)
    humidity = np.clip(humidity, 20, 95)
    
    # Create wind speed data
    wind_speed = 8 + 5 * np.sin(2 * np.pi * dates.dayofyear / 365.25) + np.random.exponential(2, n_samples)
    wind_speed = np.clip(wind_speed, 0, 30)
    
    # Create DataFrame
    df = pd.DataFrame({
        'timestamp': dates,
        'load_mw': total_load,
        'temperature': temperature,
        'humidity': humidity,
        'wind_speed': wind_speed,
        'region': np.random.choice(['North', 'South', 'Central', 'Northeast'], n_samples)
    })
    
    # Add some missing values to simulate real-world data
    missing_indices = np.random.choice(df.index, size=int(0.02 * len(df)), replace=False)
    df.loc[missing_indices, 'temperature'] = np.nan
    
    missing_indices = np.random.choice(df.index, size=int(0.01 * len(df)), replace=False)
    df.loc[missing_indices, 'humidity'] = np.nan
    
    return df

# Generate the dataset
df = generate_electrical_load_data()
print(f"Dataset shape: {df.shape}")
print(f"Date range: {df.timestamp.min()} to {df.timestamp.max()}")
df.head()

## 3. Exploratory Data Analysis (EDA)

In [3]:
# Basic data information
print("Dataset Info:")
print(df.info())
print("\nMissing values:")
print(df.isnull().sum())
print("\nBasic statistics:")
df.describe()

In [4]:
# Time series visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Load over time
axes[0,0].plot(df.timestamp, df.load_mw, alpha=0.7)
axes[0,0].set_title('Electrical Load Over Time')
axes[0,0].set_ylabel('Load (MW)')
axes[0,0].tick_params(axis='x', rotation=45)

# Temperature over time
axes[0,1].plot(df.timestamp, df.temperature, alpha=0.7, color='red')
axes[0,1].set_title('Temperature Over Time')
axes[0,1].set_ylabel('Temperature (°C)')
axes[0,1].tick_params(axis='x', rotation=45)

# Load distribution
axes[1,0].hist(df.load_mw, bins=50, alpha=0.7, edgecolor='black')
axes[1,0].set_title('Load Distribution')
axes[1,0].set_xlabel('Load (MW)')
axes[1,0].set_ylabel('Frequency')

# Load vs Temperature scatter
axes[1,1].scatter(df.temperature, df.load_mw, alpha=0.5)
axes[1,1].set_title('Load vs Temperature')
axes[1,1].set_xlabel('Temperature (°C)')
axes[1,1].set_ylabel('Load (MW)')

plt.tight_layout()
plt.show()

In [7]:
# Seasonal decomposition
df_sample = df.set_index('timestamp').resample('D').mean()  # Daily averages for decomposition

# Perform seasonal decomposition
decomposition = seasonal_decompose(df_sample['load_mw'], model='additive', period=365)

# Plot decomposition
fig, axes = plt.subplots(4, 1, figsize=(15, 12))

decomposition.observed.plot(ax=axes[0], title='Original')
decomposition.trend.plot(ax=axes[1], title='Trend')
decomposition.seasonal.plot(ax=axes[2], title='Seasonal')
decomposition.resid.plot(ax=axes[3], title='Residual')

plt.tight_layout()
plt.show()

## 4. Feature Engineering
Create additional features that can help improve load forecasting accuracy.

In [8]:
def create_time_features(df):
    """
    Create time-based features for load forecasting
    """
    df = df.copy()
    
    # Extract time components
    df['hour'] = df.timestamp.dt.hour
    df['day_of_week'] = df.timestamp.dt.dayofweek
    df['day_of_year'] = df.timestamp.dt.dayofyear
    df['month'] = df.timestamp.dt.month
    df['quarter'] = df.timestamp.dt.quarter
    df['year'] = df.timestamp.dt.year
    
    # Cyclical encoding for periodic features
    df['hour_sin'] = np.sin(2 * np.pi * df.hour / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df.hour / 24)
    
    df['day_sin'] = np.sin(2 * np.pi * df.day_of_week / 7)
    df['day_cos'] = np.cos(2 * np.pi * df.day_of_week / 7)
    
    df['month_sin'] = np.sin(2 * np.pi * df.month / 12)
    df['month_cos'] = np.cos(2 * np.pi * df.month / 12)
    
    # Binary features
    df['is_weekend'] = (df.day_of_week >= 5).astype(int)
    df['is_summer'] = df.month.isin([6, 7, 8]).astype(int)
    df['is_winter'] = df.month.isin([12, 1, 2]).astype(int)
    
    # Peak hours (typically 9-11 AM and 6-9 PM)
    df['is_peak_morning'] = df.hour.isin([9, 10, 11]).astype(int)
    df['is_peak_evening'] = df.hour.isin([18, 19, 20, 21]).astype(int)
    
    return df

def create_lag_features(df, target_col='load_mw', lags=[1, 2, 3, 24, 48, 168]):
    """
    Create lag features for time series forecasting
    """
    df = df.copy()
    
    for lag in lags:
        df[f'{target_col}_lag_{lag}'] = df[target_col].shift(lag)
    
    # Rolling statistics
    df[f'{target_col}_rolling_mean_24'] = df[target_col].rolling(window=24, min_periods=1).mean()
    df[f'{target_col}_rolling_std_24'] = df[target_col].rolling(window=24, min_periods=1).std()
    df[f'{target_col}_rolling_mean_168'] = df[target_col].rolling(window=168, min_periods=1).mean()
    
    return df

def create_weather_features(df):
    """
    Create weather-derived features
    """
    df = df.copy()
    
    # Temperature-based features
    df['temp_squared'] = df.temperature ** 2
    df['temp_deviation_from_comfort'] = np.abs(df.temperature - 22)  # 22°C as comfort zone
    
    # Heat index approximation
    df['heat_index'] = df.temperature + 0.5 * df.humidity
    
    # Weather categories
    df['temp_category'] = pd.cut(df.temperature, bins=[-float('inf'), 10, 20, 30, float('inf')], 
                                labels=['Cold', 'Cool', 'Warm', 'Hot'])
    
    return df

# Apply feature engineering
df_features = create_time_features(df)
df_features = create_lag_features(df_features)
df_features = create_weather_features(df_features)

print(f"Original features: {df.shape[1]}")
print(f"After feature engineering: {df_features.shape[1]}")
print("\nNew features created:")
new_features = set(df_features.columns) - set(df.columns)
print(list(new_features))

## 5. Data Imputation and Preprocessing

In [9]:
# Check missing values after feature engineering
print("Missing values after feature engineering:")
missing_summary = df_features.isnull().sum()
print(missing_summary[missing_summary > 0])

# Separate numerical and categorical columns
numerical_cols = df_features.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = ['region', 'temp_category']

print(f"\nNumerical columns: {len(numerical_cols)}")
print(f"Categorical columns: {len(categorical_cols)}")

In [10]:
# Imputation strategies
def impute_missing_values(df, numerical_cols, categorical_cols):
    """
    Apply different imputation strategies for missing values
    """
    df_imputed = df.copy()
    
    # For numerical columns - use KNN imputation for weather features
    weather_cols = ['temperature', 'humidity', 'wind_speed']
    weather_cols_present = [col for col in weather_cols if col in df_imputed.columns]
    
    if weather_cols_present:
        knn_imputer = KNNImputer(n_neighbors=5)
        df_imputed[weather_cols_present] = knn_imputer.fit_transform(df_imputed[weather_cols_present])
    
    # For other numerical columns with missing values - use median
    other_numerical = [col for col in numerical_cols if col not in weather_cols_present]
    
    for col in other_numerical:
        if df_imputed[col].isnull().sum() > 0:
            median_imputer = SimpleImputer(strategy='median')
            df_imputed[[col]] = median_imputer.fit_transform(df_imputed[[col]])
    
    # For categorical columns - use mode
    for col in categorical_cols:
        if col in df_imputed.columns and df_imputed[col].isnull().sum() > 0:
            mode_imputer = SimpleImputer(strategy='most_frequent')
            df_imputed[[col]] = mode_imputer.fit_transform(df_imputed[[col]])
    
    return df_imputed

# Apply imputation
df_clean = impute_missing_values(df_features, numerical_cols, categorical_cols)

# Verify no missing values remain
print("Missing values after imputation:")
print(df_clean.isnull().sum().sum())

# Recalculate weather features after imputation
df_clean = create_weather_features(df_clean)

print("Data imputation completed successfully!")

In [11]:
# Encode categorical variables
def encode_categorical_features(df, categorical_cols):
    """
    Encode categorical features using appropriate methods
    """
    df_encoded = df.copy()
    
    # One-hot encoding for region
    if 'region' in df_encoded.columns:
        region_dummies = pd.get_dummies(df_encoded['region'], prefix='region')
        df_encoded = pd.concat([df_encoded, region_dummies], axis=1)
        df_encoded.drop('region', axis=1, inplace=True)
    
    # Label encoding for ordinal categorical variables
    if 'temp_category' in df_encoded.columns:
        le = LabelEncoder()
        df_encoded['temp_category_encoded'] = le.fit_transform(df_encoded['temp_category'].astype(str))
        df_encoded.drop('temp_category', axis=1, inplace=True)
    
    return df_encoded

# Apply encoding
df_encoded = encode_categorical_features(df_clean, categorical_cols)

print(f"Shape after encoding: {df_encoded.shape}")
print("Data preprocessing completed!")

## 6. Data Splitting and Scaling

In [12]:
# Prepare data for modeling
# Remove timestamp and target variable from features
feature_cols = [col for col in df_encoded.columns if col not in ['timestamp', 'load_mw']]
target_col = 'load_mw'

# Remove rows with NaN values (mainly from lag features)
df_model = df_encoded.dropna().copy()

X = df_model[feature_cols]
y = df_model[target_col]

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"\nFeatures used: {len(feature_cols)}")

In [13]:
# Time series split for proper validation
# Use the last 20% of data for testing
split_index = int(0.8 * len(df_model))

X_train = X.iloc[:split_index]
X_test = X.iloc[split_index:]
y_train = y.iloc[:split_index]
y_test = y.iloc[split_index:]

# Get corresponding timestamps for analysis
train_timestamps = df_model['timestamp'].iloc[:split_index]
test_timestamps = df_model['timestamp'].iloc[split_index:]

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Training period: {train_timestamps.min()} to {train_timestamps.max()}")
print(f"Test period: {test_timestamps.min()} to {test_timestamps.max()}")

In [14]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test.index)

print("Feature scaling completed!")
print(f"Feature means after scaling: {X_train_scaled.mean().round(3).head()}")
print(f"Feature stds after scaling: {X_train_scaled.std().round(3).head()}")

## 7. Model Training and Validation

In [15]:
# Define evaluation metrics
def evaluate_model(y_true, y_pred, model_name):
    """
    Calculate and display evaluation metrics
    """
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"\n{model_name} Performance:")
    print(f"MAE: {mae:.2f} MW")
    print(f"RMSE: {rmse:.2f} MW")
    print(f"R²: {r2:.4f}")
    print(f"MAPE: {mape:.2f}%")
    
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'MAPE': mape}

# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# Train and evaluate models
model_results = {}
trained_models = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train model
    if name == 'Linear Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    # Evaluate model
    results = evaluate_model(y_test, y_pred, name)
    model_results[name] = results
    trained_models[name] = (model, y_pred)

print("\nModel training completed!")

In [16]:
# Cross-validation with time series split
def time_series_cv(X, y, model, n_splits=5):
    """
    Perform time series cross-validation
    """
    tscv = TimeSeriesSplit(n_splits=n_splits)
    cv_scores = []
    
    for train_idx, val_idx in tscv.split(X):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_val)
        
        mae = mean_absolute_error(y_val, y_pred)
        cv_scores.append(mae)
    
    return cv_scores

# Perform cross-validation for Random Forest (best performing model)
print("Performing time series cross-validation...")
rf_cv_scores = time_series_cv(X_train, y_train, RandomForestRegressor(n_estimators=100, random_state=42))

print(f"Cross-validation MAE scores: {[f'{score:.2f}' for score in rf_cv_scores]}")
print(f"Mean CV MAE: {np.mean(rf_cv_scores):.2f} ± {np.std(rf_cv_scores):.2f} MW")

## 8. Model Evaluation and Visualization

In [17]:
# Comparison of model performance
results_df = pd.DataFrame(model_results).T
print("Model Performance Comparison:")
print(results_df)

# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

metrics = ['MAE', 'RMSE', 'R2', 'MAPE']
for i, metric in enumerate(metrics):
    ax = axes[i//2, i%2]
    results_df[metric].plot(kind='bar', ax=ax, color=['skyblue', 'lightgreen', 'lightcoral'])
    ax.set_title(f'{metric} Comparison')
    ax.set_ylabel(metric)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [18]:
# Detailed visualization of predictions
best_model_name = 'Random Forest'  # Based on typical performance
best_model, best_predictions = trained_models[best_model_name]

# Time series plot of predictions vs actual
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Full test period
axes[0].plot(test_timestamps, y_test.values, label='Actual', alpha=0.8)
axes[0].plot(test_timestamps, best_predictions, label='Predicted', alpha=0.8)
axes[0].set_title(f'{best_model_name} Predictions vs Actual - Full Test Period')
axes[0].set_ylabel('Load (MW)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Zoomed view (first week)
week_mask = test_timestamps <= test_timestamps.min() + timedelta(days=7)
axes[1].plot(test_timestamps[week_mask], y_test[week_mask], label='Actual', marker='o', markersize=2)
axes[1].plot(test_timestamps[week_mask], best_predictions[week_mask], label='Predicted', marker='s', markersize=2)
axes[1].set_title('First Week - Detailed View')
axes[1].set_ylabel('Load (MW)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Residuals
residuals = y_test.values - best_predictions
axes[2].plot(test_timestamps, residuals, alpha=0.7, color='red')
axes[2].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[2].set_title('Prediction Residuals')
axes[2].set_ylabel('Residual (MW)')
axes[2].set_xlabel('Time')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [19]:
# Scatter plot and residual analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Predicted vs Actual scatter plot
axes[0,0].scatter(y_test, best_predictions, alpha=0.6)
axes[0,0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0,0].set_xlabel('Actual Load (MW)')
axes[0,0].set_ylabel('Predicted Load (MW)')
axes[0,0].set_title('Predicted vs Actual')
axes[0,0].grid(True, alpha=0.3)

# Residuals distribution
axes[0,1].hist(residuals, bins=50, alpha=0.7, edgecolor='black')
axes[0,1].set_xlabel('Residuals (MW)')
axes[0,1].set_ylabel('Frequency')
axes[0,1].set_title('Residuals Distribution')
axes[0,1].axvline(x=0, color='red', linestyle='--')

# QQ plot for residuals normality
from scipy import stats
stats.probplot(residuals, dist="norm", plot=axes[1,0])
axes[1,0].set_title('Q-Q Plot of Residuals')

# Residuals vs predicted values
axes[1,1].scatter(best_predictions, residuals, alpha=0.6)
axes[1,1].axhline(y=0, color='red', linestyle='--')
axes[1,1].set_xlabel('Predicted Load (MW)')
axes[1,1].set_ylabel('Residuals (MW)')
axes[1,1].set_title('Residuals vs Predicted')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Feature Importance Analysis

In [20]:
# Feature importance for Random Forest
if best_model_name == 'Random Forest':
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Plot top 20 most important features
    plt.figure(figsize=(12, 8))
    top_features = feature_importance.head(20)
    
    plt.barh(range(len(top_features)), top_features['importance'], 
             color=plt.cm.viridis(np.linspace(0, 1, len(top_features))))
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Feature Importance')
    plt.title('Top 20 Most Important Features (Random Forest)')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("Top 10 Most Important Features:")
    print(feature_importance.head(10))

## 10. Inference with New Data

In [21]:
# Create a function for making predictions on new data
def predict_load(model, scaler, new_data, model_name, feature_cols):
    """
    Make load predictions on new data
    """
    # Ensure new_data has all required features
    for col in feature_cols:
        if col not in new_data.columns:
            print(f"Warning: Missing feature {col}, setting to 0")
            new_data[col] = 0
    
    # Select and order features correctly
    X_new = new_data[feature_cols]
    
    # Scale if needed (for Linear Regression)
    if model_name == 'Linear Regression':
        X_new_scaled = scaler.transform(X_new)
        predictions = model.predict(X_new_scaled)
    else:
        predictions = model.predict(X_new)
    
    return predictions

# Generate future data for demonstration
def generate_future_data(start_date, periods=168):  # 1 week of hourly data
    """
    Generate future data with weather forecasts for demonstration
    """
    future_dates = pd.date_range(start=start_date, periods=periods, freq='H')
    
    # Simulate weather forecast (with some uncertainty)
    base_temp = 25 + 10 * np.sin(2 * np.pi * future_dates.dayofyear / 365.25)
    temperature = base_temp + np.random.normal(0, 2, periods)  # Less variation in forecast
    
    humidity = 60 + 20 * np.sin(2 * np.pi * future_dates.dayofyear / 365.25) + np.random.normal(0, 5, periods)
    humidity = np.clip(humidity, 20, 95)
    
    wind_speed = 8 + 5 * np.sin(2 * np.pi * future_dates.dayofyear / 365.25) + np.random.exponential(1.5, periods)
    wind_speed = np.clip(wind_speed, 0, 25)
    
    future_df = pd.DataFrame({
        'timestamp': future_dates,
        'temperature': temperature,
        'humidity': humidity,
        'wind_speed': wind_speed,
        'region': np.random.choice(['North', 'South', 'Central', 'Northeast'], periods)
    })
    
    return future_df

# Generate future scenario
future_start = test_timestamps.max() + timedelta(hours=1)
future_data = generate_future_data(future_start)

print(f"Generated future data for prediction:")
print(f"Period: {future_data.timestamp.min()} to {future_data.timestamp.max()}")
print(f"Shape: {future_data.shape}")

In [22]:
# Prepare future data for prediction
def prepare_future_data_for_prediction(future_data, last_known_load_values):
    """
    Prepare future data by adding all necessary features
    Note: In practice, lag features would need special handling
    """
    # Add time features
    future_processed = create_time_features(future_data)
    
    # Add weather features
    future_processed = create_weather_features(future_processed)
    
    # Encode categorical features
    future_processed = encode_categorical_features(future_processed, ['region', 'temp_category'])
    
    # For lag features, we'll use the last known values as approximation
    # In practice, you'd need a more sophisticated approach for multi-step prediction
    lag_features = [col for col in feature_cols if 'lag' in col or 'rolling' in col]
    
    for feature in lag_features:
        if 'lag_1' in feature:
            future_processed[feature] = last_known_load_values[-1]
        elif 'lag_2' in feature:
            future_processed[feature] = last_known_load_values[-2] if len(last_known_load_values) > 1 else last_known_load_values[-1]
        elif 'lag_3' in feature:
            future_processed[feature] = last_known_load_values[-3] if len(last_known_load_values) > 2 else last_known_load_values[-1]
        elif 'lag_24' in feature:
            future_processed[feature] = np.mean(last_known_load_values[-24:]) if len(last_known_load_values) >= 24 else last_known_load_values[-1]
        elif 'rolling_mean_24' in feature:
            future_processed[feature] = np.mean(last_known_load_values[-24:]) if len(last_known_load_values) >= 24 else np.mean(last_known_load_values)
        elif 'rolling_std_24' in feature:
            future_processed[feature] = np.std(last_known_load_values[-24:]) if len(last_known_load_values) >= 24 else np.std(last_known_load_values) if len(last_known_load_values) > 1 else 0
        elif 'rolling_mean_168' in feature:
            future_processed[feature] = np.mean(last_known_load_values[-168:]) if len(last_known_load_values) >= 168 else np.mean(last_known_load_values)
        else:
            future_processed[feature] = np.mean(last_known_load_values[-48:]) if len(last_known_load_values) >= 48 else np.mean(last_known_load_values)
    
    return future_processed

# Get last known load values from test set
last_known_loads = y_test.values[-168:]  # Last week of known data

# Prepare future data
future_prepared = prepare_future_data_for_prediction(future_data, last_known_loads)

# Make predictions
future_predictions = predict_load(best_model, scaler, future_prepared, best_model_name, feature_cols)

print(f"Generated {len(future_predictions)} load predictions")
print(f"Predicted load range: {future_predictions.min():.2f} - {future_predictions.max():.2f} MW")

In [23]:
# Visualize future predictions
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Historical + Future predictions
historical_period = test_timestamps[-168:]  # Last week of test data
historical_actual = y_test.iloc[-168:].values
historical_pred = best_predictions[-168:]

# Plot historical data
axes[0].plot(historical_period, historical_actual, label='Historical Actual', linewidth=2)
axes[0].plot(historical_period, historical_pred, label='Historical Predicted', linewidth=2, alpha=0.8)

# Plot future predictions
axes[0].plot(future_data['timestamp'], future_predictions, label='Future Predictions', 
            linewidth=2, linestyle='--', color='red')
axes[0].axvline(x=future_start, color='black', linestyle=':', alpha=0.7, label='Forecast Start')
axes[0].set_title('Load Forecasting: Historical Performance and Future Predictions')
axes[0].set_ylabel('Load (MW)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Future predictions only
axes[1].plot(future_data['timestamp'], future_predictions, marker='o', markersize=3, linewidth=2)
axes[1].set_title('Future Load Predictions (Next Week)')
axes[1].set_ylabel('Load (MW)')
axes[1].grid(True, alpha=0.3)

# Weather context for future period
ax2 = axes[2]
ax3 = ax2.twinx()

ax2.plot(future_data['timestamp'], future_data['temperature'], color='red', label='Temperature', linewidth=2)
ax3.plot(future_data['timestamp'], future_predictions, color='blue', label='Predicted Load', linewidth=2)

ax2.set_ylabel('Temperature (°C)', color='red')
ax3.set_ylabel('Load (MW)', color='blue')
ax2.set_title('Temperature vs Predicted Load')
ax2.grid(True, alpha=0.3)

# Add legends
lines1, labels1 = ax2.get_legend_handles_labels()
lines2, labels2 = ax3.get_legend_handles_labels()
ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.tight_layout()
plt.show()

## 11. Summary and Key Insights

In [26]:
# Summary statistics and insights
print("=" * 60)
print("REGIONAL LOAD FORECASTING WORKSHOP SUMMARY")
print("=" * 60)

print("\n📊 DATASET OVERVIEW:")
print(f"• Total samples: {len(df):,}")
print(f"• Time period: {df.timestamp.min().strftime('%Y-%m-%d')} to {df.timestamp.max().strftime('%Y-%m-%d')}")
print(f"• Features created: {len(feature_cols)}")
print(f"• Training samples: {len(X_train):,}")
print(f"• Test samples: {len(X_test):,}")

print("\n🎯 MODEL PERFORMANCE:")
for model_name, metrics in model_results.items():
    print(f"• {model_name}:")
    print(f"  - MAE: {metrics['MAE']:.2f} MW")
    print(f"  - RMSE: {metrics['RMSE']:.2f} MW")
    print(f"  - R²: {metrics['R2']:.4f}")
    print(f"  - MAPE: {metrics['MAPE']:.2f}%")

print(f"\n🏆 BEST MODEL: {best_model_name}")
best_results = model_results[best_model_name]
print(f"• Accuracy: {best_results['R2']:.1%}")
print(f"• Average error: {best_results['MAE']:.2f} MW")
print(f"• Error percentage: {best_results['MAPE']:.2f}%")

print("\n🔍 KEY INSIGHTS:")
print("• Load patterns show strong daily and weekly seasonality")
print("• Temperature is a major driver of electricity demand")
print("• Historical load values (lag features) are highly predictive")
print("• Peak demand occurs during extreme weather conditions")
print("• Weekdays generally have higher demand than weekends")

if best_model_name == 'Random Forest':
    print("\n🌟 TOP PREDICTIVE FEATURES:")
    for i, (feature, importance) in enumerate(feature_importance.head(5).values):
        print(f"• {i+1}. {feature}: {importance:.4f}")

print("\n🚀 FUTURE PREDICTIONS:")
print(f"• Forecast period: {future_data.timestamp.min().strftime('%Y-%m-%d %H:%M')} to {future_data.timestamp.max().strftime('%Y-%m-%d %H:%M')}")
print(f"• Predicted load range: {future_predictions.min():.0f} - {future_predictions.max():.0f} MW")
print(f"• Average predicted load: {future_predictions.mean():.0f} MW")

print("\n💡 RECOMMENDATIONS:")
print("• Monitor weather forecasts for load planning")
print("• Consider regional differences in load patterns")
print("• Update models regularly with new data")
print("• Implement ensemble methods for improved accuracy")
print("• Add external factors (holidays, economic indicators)")

print("\n" + "=" * 60)